In [23]:
import json
import great_expectations as ge
import pandas as pd
from datetime import datetime

In [2]:
context = ge.data_context.DataContext()

2019-12-28T15:59:23-0800 - INFO - Using project config: /Users/saurabh/Documents/Work/codebase/python_ws/ge_blog_project/great_expectations/great_expectations.yml


In [3]:
data_asset_name = "daily_revenue" # TODO: replace with your value!
normalized_data_asset_name = context.normalize_data_asset_name(data_asset_name)
normalized_data_asset_name

NormalizedDataAssetName(datasource='dbt_analytics', generator='default', generator_asset='daily_revenue')

In [4]:
expectation_suite_name = "failure" # TODO: replace with your value!

In [5]:
context.get_expectation_suite(normalized_data_asset_name, expectation_suite_name)

{'data_asset_name': 'dbt_analytics/default/daily_revenue',
 'expectation_suite_name': 'failure',
 'meta': {'great_expectations.__version__': '0.8.6'},
 'expectations': [{'expectation_type': 'expect_column_values_to_not_be_null',
   'kwargs': {'column': 'order_id'}},
  {'expectation_type': 'expect_column_values_to_be_in_set',
   'kwargs': {'column': 'order_type', 'value_set': ['online', 'retail']}},
  {'expectation_type': 'expect_column_values_to_be_between',
   'kwargs': {'column': 'order_revenue_usd',
    'min_value': 1,
    'max_value': 10000}}],
 'data_asset_type': 'Dataset'}

In [6]:
batch_kwargs = {'query': "SELECT * FROM daily_revenue WHERE load_date = '{}'".format('2019-12-02')}
batch = context.get_batch(normalized_data_asset_name, expectation_suite_name, batch_kwargs)
batch.head()

2019-12-28T16:01:19-0800 - INFO - 	3 expectation(s) included in expectation_suite.


,order_id,user_id,order_revenue_usd,order_type,load_date
0,201912013,1003,500,retail,2019-12-02
1,201912014,1004,0,zzz_corrupt_value_zzz,2019-12-02


In [10]:
run_id = datetime.utcnow().isoformat().replace(":", "") + "Z"
run_id

'2019-12-29T000211.765938Z'

In [21]:
validation_result = batch.validate(run_id=run_id)

if validation_result["success"]:
    print("This data meets all expectations for {}".format(str(data_asset_name)))
else:
    print("This data is not a valid batch of {}".format(str(data_asset_name)))

2019-12-28T16:12:47-0800 - INFO - 	3 expectation(s) included in expectation_suite.
This data is not a valid batch of daily_revenue


In [15]:
print(json.dumps(validation_result, indent=4))

{
    "results": [
        {
            "success": true,
            "result": {
                "element_count": 2,
                "unexpected_count": 0.0,
                "unexpected_percent": 0.0,
                "partial_unexpected_list": []
            },
            "expectation_config": {
                "expectation_type": "expect_column_values_to_not_be_null",
                "kwargs": {
                    "column": "order_id"
                }
            },
            "exception_info": {
                "raised_exception": false,
                "exception_message": null,
                "exception_traceback": null
            }
        },
        {
            "success": false,
            "result": {
                "element_count": 2,
                "missing_count": 0.0,
                "missing_percent": 0.0,
                "unexpected_count": 1.0,
                "unexpected_percent": 50.0,
                "unexpected_percent_nonmissing": 50.0,
                "pa

In [22]:
if not validation_result["success"]:
    raise ValueError("This data is not a valid batch of {}".format(str(data_asset_name)))

ValueError: This data is not a valid batch of daily_revenue

In [16]:
#context.open_data_docs()

In [17]:
#context.build_data_docs()

{'local_site': '/Users/saurabh/Documents/Work/codebase/python_ws/ge_blog_project/great_expectations/uncommitted/data_docs/local_site/index.html'}

In [19]:
# results = context.run_validation_operator(
#     assets_to_validate=[batch],
#     run_id=run_id,
#     validation_operator_name="action_list_operator",
# )

2019-12-28T16:06:47-0800 - INFO - 	3 expectation(s) included in expectation_suite.
